In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from collection_cache import CollectionCache
from pymongo import IndexModel, ASCENDING, DESCENDING
from mongo_db import MongoDB
from utils import load_location_data

In [8]:
cache = CollectionCache()
locations = load_location_data()
cache.collection_complete(locations)

True

In [21]:
def date_to_str(date):
    date = str(date)
    return '_'.join(date.split(' '))

In [24]:
start = datetime(2022,1,1,0,0,0)
end = datetime(2022,12,31,23,0,0)

dates = list(pd.date_range(start,end,freq='H'))

dates = np.vectorize(date_to_str)(dates)

In [42]:
mongo = MongoDB()
totals = []

for lat, lon in tqdm(locations):
    totals.append(mongo.collection.count_documents({"lat": lat, "lon": lon}))

100%|██████████| 450/450 [00:38<00:00, 11.66it/s]


In [38]:
mongo.collection.create_index([("lat", DESCENDING),("lon", ASCENDING)])

'lat_-1_lon_1'

In [44]:
Counter(totals)

Counter({8161: 450})

In [55]:
from tqdm.contrib import itertools

In [110]:
import pickle

def save_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)

In [111]:
results = defaultdict(set)

counter = 0
for loc, date in itertools.product(locations[:10], dates):
    lat = loc[0]
    lon = loc[1]
    key = f"{lat}_{lon}_{date}"
    num_doc = len(list(mongo.collection.find({"_id": key}).limit(1)))
    if num_doc == 0:
        results[f"{lat}_{lon}"].add(date)
    if counter % 1000 == 0:
        save_pickle(results, '../data_analysis.pickle')
    counter += 1

  0%|          | 0/87600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [112]:
results

defaultdict(set,
            {'34.114_-118.407': {'2022-03-13_02:00:00',
              '2022-11-06_02:00:00',
              '2022-11-06_03:00:00',
              '2022-11-06_04:00:00',
              '2022-11-06_05:00:00',
              '2022-11-06_06:00:00',
              '2022-11-06_07:00:00',
              '2022-11-06_08:00:00',
              '2022-11-06_09:00:00',
              '2022-11-06_10:00:00',
              '2022-11-06_11:00:00',
              '2022-11-06_12:00:00',
              '2022-11-06_13:00:00',
              '2022-11-06_14:00:00',
              '2022-11-06_15:00:00',
              '2022-11-06_16:00:00',
              '2022-11-06_17:00:00',
              '2022-11-06_18:00:00',
              '2022-11-06_19:00:00',
              '2022-11-06_20:00:00',
              '2022-11-06_21:00:00',
              '2022-11-06_22:00:00',
              '2022-11-06_23:00:00',
              '2022-11-07_00:00:00',
              '2022-11-07_01:00:00',
              '2022-11-07_02:00:00',
  

In [114]:
mongo.collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'lat_-1_lon_1': {'v': 2, 'key': [('lat', -1), ('lon', 1)]}}

In [115]:
results.keys()

dict_keys(['34.114_-118.407', '37.756_-122.443', '32.831_-117.122', '33.938_-117.395'])

In [3]:
from spark_cache import SparkCache

In [78]:
locations = load_location_data()

spark_cache = SparkCache()

files = []

for lat, lon in tqdm(locations):
    file = f"data/vc_{lat}_{lon}_2022_11.csv"
    spark_cache.add_file(file)
    files.append(file)


100%|██████████| 450/450 [05:10<00:00,  1.45it/s]


In [5]:
from airflow_dag import process_data_with_spark_store_in_mongo

In [79]:
process_data_with_spark_store_in_mongo()

[2023-02-28 15:16:13,154] {airflow_dag.py:96} INFO - (0) Processing: gcp_filename = 'data/vc_34.114_-118.407_2022_11.csv'


[2023-02-28 15:16:19,758] {airflow_dag.py:96} INFO - (1) Processing: gcp_filename = 'data/vc_37.756_-122.443_2022_11.csv'
[2023-02-28 15:16:22,653] {airflow_dag.py:96} INFO - (2) Processing: gcp_filename = 'data/vc_32.831_-117.122_2022_11.csv'


[2023-02-28 15:16:26,177] {airflow_dag.py:96} INFO - (3) Processing: gcp_filename = 'data/vc_33.938_-117.395_2022_11.csv'


[2023-02-28 15:16:31,041] {airflow_dag.py:96} INFO - (4) Processing: gcp_filename = 'data/vc_38.567_-121.468_2022_11.csv'


[2023-02-28 15:16:35,626] {airflow_dag.py:96} INFO - (5) Processing: gcp_filename = 'data/vc_37.302_-121.849_2022_11.csv'


...

[2023-02-28 15:47:27,261] {airflow_dag.py:96} INFO - (444) Processing: gcp_filename = 'data/vc_36.024_-120.907_2022_11.csv'
[2023-02-28 15:47:31,224] {airflow_dag.py:96} INFO - (445) Processing: gcp_filename = 'data/vc_38.256_-119.212_2022_11.csv'


[2023-02-28 15:47:34,728] {airflow_dag.py:96} INFO - (446) Processing: gcp_filename = 'data/vc_35.277_-116.072_2022_11.csv'


[2023-02-28 15:47:42,069] {airflow_dag.py:96} INFO - (447) Processing: gcp_filename = 'data/vc_37.452_-120.704_2022_11.csv'


[2023-02-28 15:47:46,783] {airflow_dag.py:96} INFO - (448) Processing: gcp_filename = 'data/vc_34.634_-120.116_2022_11.csv'


[2023-02-28 15:47:51,133] {airflow_dag.py:96} INFO - (449) Processing: gcp_filename = 'data/vc_35.941_-121.08_2022_11.csv'


In [16]:
import os

In [64]:
file_path = "gs://msds697-solar-prod/data/vc_32.628_-117.014_2022_10.csv"

def load_csv(file_path):
    gcs_key = os.environ['GS_SERVICE_ACCOUNT_KEY_FILE']
    df1 = pd.read_csv(file_path, storage_options={"token": gcs_key})
    return df1

col_order = list(load_csv(file_path).columns)

In [76]:
def remove_duplicates(file_path):
    df_new = load_csv(file_path).groupby(['lat','lon','date','time']) \
                                .first().reset_index()[col_order]
    df_new.to_csv('../nov_data/' + file_path.split('/')[-1], index=False)


In [77]:
base = "gs://msds697-solar-prod/"

for lat, lon in tqdm(locations):
    file = f"data/vc_{lat}_{lon}_2022_11.csv"
    remove_duplicates(base + file)

100%|██████████| 450/450 [02:42<00:00,  2.77it/s]


In [80]:
mongo = MongoDB()

len(list(mongo.find({'date':{"$regex": "^2022-11-\d\d$"}}, {})))

324000